In [1]:
import torch
import os
import json

In [2]:
os.environ['HF_HOME'] =  "/mnt/swordfish-pool2/models/"
# os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
from transformers import AutoTokenizer, LlamaForCausalLM, GenerationConfig

/home/a.saakyan/tmp/ENTER/envs/alpaca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# MPT-7B

In [9]:
import transformers
dir_of_hf_w = "mosaicml/mpt-7b-instruct"
print("Loading the model and tokenizer...")
config = transformers.AutoConfig.from_pretrained(dir_of_hf_w, trust_remote_code=True)
# config.attn_config['attn_impl'] = 'triton'
# config.init_device = 'cuda:0,1,2,3' # For fast initialization directly on GPU!
# config.init_device = 'cuda:3'

model = transformers.AutoModelForCausalLM.from_pretrained(
  dir_of_hf_w,
  config=config,
  # torch_dtype=torch.bfloat16, # Load model weights in bfloat16
  trust_remote_code=True,
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Loading the model and tokenizer...


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.13s/it]


MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [ ]:
out_dir = "../data/gyafc_w_ICHF_alpaca/informal_to_formal"
assert os.path.isdir(out_dir)
input_json = "../data/gyafc_w_ICHF_alpaca/informal_to_formal/test.json"
assert os.path.isfile(input_json)
train_file_path = "../data/gyafc_w_ICHF_alpaca/informal_to_formal/train.json"
assert os.path.isfile(train_file_path)

# fs_prompt = construct_few_shot_prompt(train_file_path, n=3)
# print(generate_prompt(input="test", fs_prompt=fs_prompt))

run_inference(input_json, out_dir, 
              "mpt", model, tokenizer, 
              n_shots=1, train_file_path=train_file_path,
              verbose=False, stop_idx=10000)

In [ ]:
out_dir = "../data/gyafc_w_ICHF_alpaca/formal_to_informal"
assert os.path.isdir(out_dir)
input_json = "../data/gyafc_w_ICHF_alpaca/formal_to_informal/test.json"
assert os.path.isfile(input_json)
train_file_path = "../data/gyafc_w_ICHF_alpaca/formal_to_informal/train.json"
assert os.path.isfile(train_file_path)

run_inference(input_json, out_dir, 
              "mpt", model, tokenizer, 
              n_shots=1, train_file_path=train_file_path,
              verbose=False, stop_idx=10000)